[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/recommendation/product-recommender/product_recommender.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/recommendation/product-recommender/product_recommender.ipynb)

# Product Recommendation Engine

Learn how to build a product recommendation engine using collaborative filtering and Pinecone.

In this example, we will generate product recommendations for ecommerce customers based on previous orders and trending items. This example covers preparing the vector embeddings, creating and deploying the Pinecone service, writing data to Pinecone, and finally querying Pinecone to receive a ranked list of recommended products.

## Data Preparation

**Import Python Libraries**

In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools

**Load the (Example) Instacart Data**

We are going to use the [Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/data) dataset for this task.

The data used throughout this example is a set of files describing customers' orders over time. The main focus is on the *orders.csv* file, where each line represents a relation between a user and the order. In other words, each line has information on *userid* (user who made the order) and *orderid*. Note there is no information about products in this table. Product information related to specific orders is stored in the *order_product__*.csv* dataset.

In [ ]:
order_products_train = pd.read_csv('data/order_products__train.csv')
order_products_prior = pd.read_csv('data/order_products__prior.csv')
products = pd.read_csv('data/products.csv')
orders = pd.read_csv('data/orders.csv')

order_products = order_products_train.append(order_products_prior)

**Preparing data for the model**


The Collaborative Filtering model used in this example requires only users’ historical preferences on a set of items. As there is no explicit rating in the data we are using, the purchase quantity can represent a “confidence” in terms of how strong the interaction was between the user and the products.

The dataframe data will store this data and will be the base for the model.

In [ ]:
customer_order_products = pd.merge(orders, order_products, how='inner',on='order_id')

# creating a table with "confidences"
data = customer_order_products.groupby(['user_id', 'product_id'])[['order_id']].count().reset_index()
data.columns=["user_id", "product_id", "total_orders"]
data.product_id = data.product_id.astype('int64')

# Create a lookup frame so we can get the product names back in readable form later.
products_lookup = products[['product_id', 'product_name']].drop_duplicates()
products_lookup['product_id'] = products_lookup.product_id.astype('int64')

We will create three prototype users here and add them to our data dataframe. Each user will be buying only a specific product:
- The first user will be buying only **Mineral Water**
- The second user will be buying baby products: **No More Tears Baby Shampoo** and **Baby Wash & Shampoo**

These users will be later used for querying and examination of the model results.

In [ ]:
data_new = pd.DataFrame([[data.user_id.max() + 1, 22802, 97],
                         [data.user_id.max() + 2, 26834, 89],
                         [data.user_id.max() + 2, 12590, 77]
                        ], columns=['user_id', 'product_id', 'total_orders'])
data_new

,user_id,product_id,total_orders
0,206210,22802,97
1,206211,26834,89
2,206211,12590,77


In [ ]:
data = data.append(data_new).reset_index(drop = True)
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


In the next step, we will first extract user and item unique ids, in order to create a CSR (Compressed Sparse Row) matrix. 


In [ ]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.total_orders)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((purchases, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((purchases, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

## Implicit Model

In this section we will demonstrate creation and training of a recommender model using the **implicit** library. The recommendation model is based off the algorithms described in the paper [Collaborative Filtering for Implicit Feedback Datasets](https://www.researchgate.net/publication/220765111_Collaborative_Filtering_for_Implicit_Feedback_Datasets) with performance optimizations described in [Applications of the Conjugate Gradient Method for Implicit Feedback Collaborative Filtering](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.6473&rep=rep1&type=pdf).


In [ ]:
!pip install --quiet -U implicit

In [ ]:
import implicit
from implicit import evaluation

#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_product_user, train_percentage=0.9)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=50,
                                             num_threads=1)

alpha_val = 15
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/50 [00:00<?, ?it/s]

We will evaluate the model using the inbuilt library function

In [ ]:
test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set.T, test_set.T, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/206212 [00:00<?, ?it/s]

{'precision': 0.27489359984895717,
 'map': 0.04460861877969595,
 'ndcg': 0.14436536385146576,
 'auc': 0.6551648380086259}

This is what item and user factors look like. These vectors will be stored in our vector index later and used for recommendation.

In [ ]:
model.item_factors[1:3]

array([[ 0.01009897,  0.00260342,  0.00165942,  0.01748168,  0.00649343,
        -0.01647822,  0.01860397, -0.01009837,  0.01125452,  0.01987451,
        -0.00579512,  0.00421128,  0.01707346, -0.00212536,  0.01915585,
         0.03640049, -0.01142028,  0.01023709,  0.00446458, -0.00143529,
        -0.00024208,  0.00909473, -0.01408565,  0.02619351,  0.00210135,
        -0.00378899,  0.01231347,  0.00278133,  0.00071992,  0.00915809,
         0.01640408,  0.00880539, -0.00648519, -0.01160682,  0.00664212,
        -0.00406996,  0.01543106,  0.00690582,  0.00898032,  0.00277333,
         0.00626428, -0.01610408,  0.01018737,  0.0008459 ,  0.02026955,
        -0.01055363, -0.00107795,  0.01484767,  0.01800155, -0.00275021,
        -0.0018283 , -0.00346971,  0.00077051, -0.01080908,  0.00037001,
        -0.00290308,  0.00491365, -0.01362148, -0.00129594,  0.00192484,
         0.00101756, -0.00051836,  0.00603317,  0.01611738,  0.00511096,
        -0.0053055 ,  0.01907502,  0.01232757,  0.0

In [ ]:
model.user_factors[1:3]

array([[ 7.24285245e-01,  5.59004486e-01,  4.96992081e-01,
        -4.15437818e-01, -1.94785964e+00, -2.23764396e+00,
        -1.76767483e-02, -2.21530461e+00, -6.52559578e-01,
         2.78620571e-01,  6.03808701e-01,  1.27670407e-01,
         3.06052566e-01, -9.93388355e-01, -5.34315288e-01,
         1.20948291e+00, -2.11217976e+00,  1.67127061e+00,
         1.03314137e+00,  8.54326487e-01,  1.85733151e+00,
         5.69297194e-01, -8.93577933e-01,  1.76394248e+00,
         1.28939009e+00,  3.32375497e-01, -2.60327369e-01,
         4.21450347e-01, -1.72091925e+00,  1.10491872e+00,
        -1.86411276e-01, -3.51959467e-02, -1.41517222e+00,
        -9.19971287e-01,  4.63204056e-01, -4.07809407e-01,
         1.23038590e+00, -8.25872004e-01, -1.50579488e+00,
         8.65903348e-02, -7.29649186e-01, -5.21384776e-01,
         1.59157085e+00, -8.51297379e-01,  2.81686401e+00,
        -8.55669677e-01, -3.48052949e-01, -5.16085029e-01,
         8.01080287e-01,  1.04207866e-01, -2.72860657e-0

## Configure Pinecone

Install and setup Pinecone

In [ ]:
!pip install --quiet -U pinecone-client

In [ ]:
import pinecone

In [ ]:
# Load Pinecone API key
api_key = os.getenv('PINECONE_API_KEY') or 'YOUR_API_KEY'
# Set Pinecone environment. Find next to API key in console
env = os.getenv('PINECONE_ENVIRONMENT') or "YOUR_ENV"

pinecone.init(api_key=api_key, environment=env)

[Get a Pinecone API key](http://app.pinecone.io/) if you don't have one.

In [ ]:
#List all present indexes associated with your key, should be empty on the first run
pinecone.list_indexes()

[]

**Create an Index**

In [ ]:
# Set a name for your index
index_name = 'shopping-cart-demo'

In [ ]:
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100)

**Connect to the new index**

In [ ]:
index = pinecone.Index(index_name=index_name)

## Load Data

Uploading all items (products that one can buy) and displaying some examples of products and their vector representations.


In [ ]:
# Get all of the items
all_items = [title for title in products_lookup['product_name']]

# Transform items into factors
items_factors = model.item_factors

# Prepare item factors for upload
items_to_insert = list(zip(all_items, items_factors[1:].tolist()))
display(items_to_insert[:2])

[('Chocolate Sandwich Cookies',
  [0.010098974220454693,
   0.0026034200564026833,
   0.0016594183398410678,
   0.017481675371527672,
   0.006493427790701389,
   -0.016478220000863075,
   0.018603969365358353,
   -0.010098369792103767,
   0.01125451922416687,
   0.019874505698680878,
   -0.005795117933303118,
   0.00421128049492836,
   0.017073458060622215,
   -0.0021253626327961683,
   0.019155845046043396,
   0.036400485783815384,
   -0.01142028160393238,
   0.010237086564302444,
   0.004464581608772278,
   -0.0014352924190461636,
   -0.00024208369723055512,
   0.009094727225601673,
   -0.014085653237998486,
   0.02619350701570511,
   0.002101349411532283,
   -0.0037889881059527397,
   0.012313470244407654,
   0.002781332703307271,
   0.0007199185783974826,
   0.009158086962997913,
   0.016404075548052788,
   0.008805392310023308,
   -0.006485185585916042,
   -0.01160681527107954,
   0.006642122287303209,
   -0.004069960676133633,
   0.015431062318384647,
   0.006905817426741123,
   

**Insert items into the index**

In [ ]:
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [ ]:
print('Index statistics before upsert:', index.describe_index_stats())

for e, batch in enumerate(chunks([(ii[:64],x) for ii,x in items_to_insert])):
    index.upsert(vectors=batch)

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 0, 'namespaces': {}}
Index statistics after upsert: {'dimension': 100, 'namespaces': {'': {'vector_count': 49677}}}


This is a helper method for analysing recommendations later.
This method returns top N products that someone bought in the past (based on product quantity).

In [ ]:
def products_bought_by_user_in_the_past(user_id: int, top: int = 10):

    selected = data[data.user_id == user_id].sort_values(by=['total_orders'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products_lookup.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'total_orders']].reset_index(drop=True)
    if selected.shape[0] < top:
        return selected

    return selected[:top]

In [ ]:
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


## Query for Recommendations

We are now retrieving user factors for users that we have manually created before for testing purposes. Besides these users, we are adding a random existing user. We are also displaying these users so you can see what these factors look like.

In [ ]:
user_ids = [206210, 206211, 103593]
user_factors = model.user_factors[user_to_index[user_ids]]

display(user_factors[1:])

array([[-2.446773  , -0.62870413, -0.9166386 , -1.0933994 ,  0.9897131 ,
        -2.166681  ,  0.09873585,  1.1049409 ,  1.6753025 ,  1.5794269 ,
         1.8142459 ,  1.5048354 ,  0.7157051 , -0.7888281 ,  0.06156079,
        -1.6539581 , -0.15790005,  0.5999737 , -1.4803663 , -0.03179923,
         0.91451246,  0.14260213, -1.1541293 , -0.01566206, -1.3449577 ,
        -2.232925  , -0.88052607,  0.19183849,  0.3109626 ,  1.32479   ,
         0.16483077, -0.8045166 ,  1.36922   ,  0.81774026,  1.3368418 ,
         2.8871357 ,  2.4540865 , -1.908394  ,  2.8208447 , -1.3499558 ,
        -0.90089166,  1.0632626 ,  1.8107275 , -0.83986664,  1.1764408 ,
        -1.6621239 , -1.4636188 , -2.3367987 , -1.2510904 ,  0.4349534 ,
         0.08233324,  1.0688674 , -0.41190436,  1.6045849 , -2.3667567 ,
        -1.8557758 , -0.1931467 ,  0.10383442,  1.3932719 ,  1.3465406 ,
        -0.17274773,  0.41542327, -1.0992794 ,  1.7954347 , -0.9157203 ,
        -0.3183454 ,  0.7724282 , -0.5658835 ,  1.0

### Model recommendations

We will now retrieve recommendations from our model directly, just to have these results as a baseline.

In [ ]:
print("Model recommendations\n")

start_time = time.process_time()
recommendations0 = model.recommend(userid=user_ids[0], user_items=sparse_user_product)
recommendations1 = model.recommend(userid=user_ids[1], user_items=sparse_user_product)
recommendations2 = model.recommend(userid=user_ids[2], user_items=sparse_user_product)
print("Time needed for retrieving recommended products: " + str(time.process_time() - start_time) + ' seconds.\n')

print('\nRecommendations for person 0:')
for recommendation in recommendations0:
    product_id = recommendation[0]
    print(products_lookup[products_lookup.product_id == product_id]['product_name'].values)

print('\nRecommendations for person 1:')
for recommendation in recommendations1:
    product_id = recommendation[0]
    print(products_lookup[products_lookup.product_id == product_id]['product_name'].values)

print('\nRecommendations for person 2:')
for recommendation in recommendations2:
    product_id = recommendation[0]
    print(products_lookup[products_lookup.product_id == product_id]['product_name'].values)

Model recommendations

Time needed for retrieving recommended products: 0.0625 seconds.


Recommendations for person 0:
['Sparkling Water']
['Soda']
['Smartwater']
['Zero Calorie Cola']
['Natural Artesian Water']
['Natural Spring Water']
['Distilled Water']
['Sparkling Natural Mineral Water']
['Spring Water']
['Drinking Water']

Recommendations for person 1:
['Baby Wipes Sensitive']
['YoKids Squeezers Organic Low-Fat Yogurt, Strawberry']
['Organic Blackberries']
['Organic Whole Milk']
['Eggo Pancakes Minis']
['Natural California Raisins Mini Snack Boxes']
['100% Raw Coconut Water']
['White Buttermints']
['Danimals Strawberry Explosion Flavored Smoothie']
['Strawberry Explosion/Banana Split Smoothie']

Recommendations for person 2:
['Organic Golden Delicious Apple']
['Organic Red Delicious Apple']
['Bartlett Pears']
['Organic Blackberries']
['Bag of Organic Bananas']
['Black Seedless Grapes']
['Organic Braeburn Apple']
['Organic Blueberries']
["Organic D'Anjou Pears"]
['White Peach']


### Query the index

Let's now query the index to check how quickly we retrieve results. Please note that query speed depends in part on your internet connection.

In [ ]:
# Query by user factors

start_time = time.process_time()
query_results = index.query(queries=user_factors[:-1].tolist(), top_k=10)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(user_ids, query_results.results):
    print(f'user_id={_id}')
    df = pd.DataFrame(
        {
            'products': [match.id for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_user_in_the_past(_id, top=15))

Time needed for retrieving recommended products using Pinecone: 0.03125 seconds.

user_id=206210
Recommendation: 


,products,scores
0,Mineral Water,0.919242
1,Zero Calorie Cola,0.716640
2,Orange & Lemon Flavor Variety Pack Sparkling F...,0.631119
3,Sparkling Water,0.603575
4,Milk Chocolate Almonds,0.577868
5,Extra Fancy Unsalted Mixed Nuts,0.577714
6,Popcorn,0.565397
7,Organic Coconut Water,0.547605
8,Drinking Water,0.542832
9,Tall Kitchen Bag With Febreze Odor Shield,0.538533


Top buys from the past: 


,product_id,product_name,total_orders
0,22802,Mineral Water,97


user_id=206211
Recommendation: 


,products,scores
0,Baby Wash & Shampoo,0.731054
1,No More Tears Baby Shampoo,0.695655
2,Size 6 Baby Dry Diapers,0.526953
3,Natural Applesauce Snack & Go Pouches,0.478145
4,White Buttermints,0.475006
5,Size 5 Cruisers Diapers Super Pack,0.474203
6,Go-Gurt SpongeBob SquarePants Strawberry Ripti...,0.461982
7,Baby Wipes Sensitive,0.461840
8,Original Detergent,0.456813
9,Stage 1 Newborn Hypoallergenic Liquid Detergent,0.456143


Top buys from the past: 


,product_id,product_name,total_orders
0,26834,No More Tears Baby Shampoo,89
1,12590,Baby Wash & Shampoo,77


*Note* The inference using Pinecone is much faster compared to retrieving recommendations from a model directly. Please note that this result depends on your internet connection as well. 

All that’s left to do is surface these recommendations on the shopping site, or feed them into other applications.

## Clean up

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.

In [ ]:
pinecone.delete_index(index_name)

## Summary

In this example we used [Pinecone](https://www.pinecone.io/) to build and deploy a product recommendation engine that uses collaborative filtering, relatively quickly.

Once deployed, the product recommendation engine can index new data, retrieve recommendations in milliseconds, and send results to production applications.